In [42]:
import pandas as pd
import re

In [43]:
!pip install openpyxl


[notice] A new release of pip is available: 25.1.1 -> 25.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [82]:
df=pd.read_excel("PAN Number Validation Dataset.xlsx")

In [83]:
total_records=len(df)
total_records

10000

In [84]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 1 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   Pan_Numbers  9035 non-null   object
dtypes: object(1)
memory usage: 78.3+ KB


In [85]:
df["Pan_Numbers"]=df["Pan_Numbers"].astype("string")

In [86]:
df["Pan_Numbers"]=df["Pan_Numbers"].str.strip().str.upper()

In [87]:
df[df["Pan_Numbers"]==""]

,Pan_Numbers
5019,
5020,


In [88]:
df[df["Pan_Numbers"].isna()]

,Pan_Numbers
5022,<NA>
5027,<NA>
5033,<NA>
5043,<NA>
5057,<NA>
...,...
9961,<NA>
9972,<NA>
9986,<NA>
9987,<NA>


In [89]:
df=df.replace({"Pan_Numbers":""},pd.NA)

In [90]:
df[df["Pan_Numbers"]==""]

,Pan_Numbers


In [91]:
df[df["Pan_Numbers"].isna()]

,Pan_Numbers
5019,<NA>
5020,<NA>
5022,<NA>
5027,<NA>
5033,<NA>
...,...
9961,<NA>
9972,<NA>
9986,<NA>
9987,<NA>


In [92]:
df=df.dropna(subset="Pan_Numbers")

In [93]:
df[df["Pan_Numbers"].isna()]

,Pan_Numbers


In [94]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 9033 entries, 0 to 9999
Data columns (total 1 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   Pan_Numbers  9033 non-null   string
dtypes: string(1)
memory usage: 141.1 KB


In [95]:
df["Pan_Numbers"].nunique()

9025

In [96]:
df=df.drop_duplicates(subset="Pan_Numbers",keep="first")

In [97]:
df["Pan_Numbers"].nunique()

9025

In [98]:
def adjacent_repitition(pan):
    for i in range(len(pan)-1):
        if pan[i]==pan[i+1]:
            return True
    return False

In [99]:
print(adjacent_repitition('AABCD'))
print(adjacent_repitition('FGHHH'))
print(adjacent_repitition('ABCDX'))
print(adjacent_repitition('MNJPQ'))

True
True
False
False


In [100]:
def is_sequential(pan):
    for i in range(len(pan) - 1):
        if ord(pan[i+1]) - ord(pan[i]) != 1:
            return False
    return True


In [101]:
print(is_sequential('ABCDE'))
print(is_sequential('MNOPQ'))
print(is_sequential('ABCXY'))
print(is_sequential('XYZAB'))

True
True
False
False


In [102]:
def is_valid_pan(pan):
    if len(pan)!=10:
        return False
    if not re.match(r'^[A-Z]{5}[0-9]{4}[A-Z]$',pan):
        return False
    if adjacent_repitition(pan):
        return False
    if is_sequential(pan):
        return False
    return True
    

In [103]:
df["Status"]=df["Pan_Numbers"].apply(lambda x: "Valid" if is_valid_pan(x) else "invalid")
df.head()

,Pan_Numbers,Status
0,VGLOD3180G,Valid
1,PHOXD7232L,Valid
2,MGEPH6532A,Valid
3,JJCHK4574O,invalid
4,XTQIJ2330L,invalid


In [104]:
valid_cnt=(df["Status"]=="Valid").sum()
invalid_cnt=(df["Status"]=="invalid").sum()
missing_cnt=total_records-(valid_cnt+invalid_cnt)

In [105]:
print('Total records =',total_records)
print('Valid =',valid_cnt)
print('Invalid =',invalid_cnt)
print('Missing =',missing_cnt)

Total records = 10000
Valid = 3193
Invalid = 5832
Missing = 975
